In [1]:
import numpy as np
import pandas as pd
import requests
import json
import urllib
import tensorflow as tf
import tensorflow_datasets
from tensorflow import keras
from gensim.models.callbacks import CallbackAny2Vec
%config Completer.use_jedi =False
from gensim.models import Word2Vec

In [2]:
embedded_picks=np.load('embedding_array_25.npy')
picks_array=np.load('picks_array.npy')
names_array=np.load('names_array.npy')

In [3]:
names_array.shape

(34877, 10)

In [4]:
picks_array

array([[ 85, 100,  66, ...,  94,   5,  16],
       [117, 118,  95, ...,   3,  23,  43],
       [ 85, 100,  12, ...,  51,  58,   5],
       ...,
       [116,  61,  27, ...,  41,  98,  14],
       [ 72,  22,  85, ...,  40, 112,  95],
       [ 99,  27,  61, ...,  42,   9, 119]])

In [5]:
test_start=embedded_picks.shape[0]-int(0.05*embedded_picks.shape[0])
cv_start=test_start-int(0.1*embedded_picks.shape[0])
test_picks=embedded_picks[test_start:]
testlabel_picks=picks_array[test_start:]
cv_picks=embedded_picks[cv_start:test_start]
cvlabel_picks=picks_array[cv_start:test_start]
train_picks=embedded_picks[:cv_start]
tlabel_picks=picks_array[:cv_start]
tlabel_picks=tlabel_picks.reshape((tlabel_picks.shape[0],tlabel_picks.shape[1],1))
one_labels=tf.one_hot(tlabel_picks,depth=120)
one_labels=tf.reshape(one_labels,(tlabel_picks.shape[0],tlabel_picks.shape[1],120))
one_cvlabels=tf.one_hot(cvlabel_picks,depth=120)
one_cvlabels=tf.reshape(one_cvlabels,(cvlabel_picks.shape[0],cvlabel_picks.shape[1],120))
one_testlabels=tf.one_hot(testlabel_picks,depth=120)
one_testlabels=tf.reshape(one_testlabels,(testlabel_picks.shape[0],testlabel_picks.shape[1],120))
hero_df=pd.read_csv('hero_df.csv')
hero_dict={}
for i in hero_df['sid']:
    hero_dict[i]=hero_df[hero_df['sid']==i]['localized_name'].values[0]
embed_df=pd.read_csv('hero_to_embedding.csv')
embed_df.set_index('Heroes',inplace=True)

In [6]:
hero_dict

{0: 'Anti-Mage',
 1: 'Axe',
 2: 'Bane',
 3: 'Bloodseeker',
 4: 'Crystal Maiden',
 5: 'Drow Ranger',
 6: 'Earthshaker',
 7: 'Juggernaut',
 8: 'Mirana',
 9: 'Morphling',
 10: 'Shadow Fiend',
 11: 'Phantom Lancer',
 12: 'Puck',
 13: 'Pudge',
 14: 'Razor',
 15: 'Sand King',
 16: 'Storm Spirit',
 17: 'Sven',
 18: 'Tiny',
 19: 'Vengeful Spirit',
 20: 'Windranger',
 21: 'Zeus',
 22: 'Kunkka',
 23: 'Lina',
 24: 'Lion',
 25: 'Shadow Shaman',
 26: 'Slardar',
 27: 'Tidehunter',
 28: 'Witch Doctor',
 29: 'Lich',
 30: 'Riki',
 31: 'Enigma',
 32: 'Tinker',
 33: 'Sniper',
 34: 'Necrophos',
 35: 'Warlock',
 36: 'Beastmaster',
 37: 'Queen of Pain',
 38: 'Venomancer',
 39: 'Faceless Void',
 40: 'Wraith King',
 41: 'Death Prophet',
 42: 'Phantom Assassin',
 43: 'Pugna',
 44: 'Templar Assassin',
 45: 'Viper',
 46: 'Luna',
 47: 'Dragon Knight',
 48: 'Dazzle',
 49: 'Clockwerk',
 50: 'Leshrac',
 51: "Nature's Prophet",
 52: 'Lifestealer',
 53: 'Dark Seer',
 54: 'Clinkz',
 55: 'Omniknight',
 56: 'Enchantress'

In [6]:
def pipe(arr,labels):
    data=tf.data.Dataset.from_tensor_slices(arr)
    vals=tf.data.Dataset.from_tensor_slices(labels)
    combined=tf.data.Dataset.zip((data,vals))
    finale=combined.map(lambda x,y: (x[:-1],y[-1]))
    return finale
def preds(arr):
    if len(arr.shape)!=2:
        data=tf.data.Dataset.from_tensor_slices(arr)
        data=data.map(lambda x: x[:-1])
        data=data.batch(1,drop_remainder=True)
    else:
        data=np.reshape(arr,(1,arr.shape[0],arr.shape[1]))
        data=tf.data.Dataset.from_tensor_slices(data)
        data=data.map(lambda x: x[:-1])
        data=data.batch(1,drop_remainder=True)
    return data

In [7]:
train=pipe(train_picks,one_labels)
train=train.batch(32,drop_remainder=True)
cv=pipe(cv_picks,one_cvlabels)
cv=cv.batch(1,drop_remainder=True)
test=pipe(test_picks,one_testlabels)
test=test.batch(1,drop_remainder=True)

In [10]:
test_start-cv_start

3487

In [8]:
model=keras.models.Sequential([
    keras.layers.LSTM(128,recurrent_dropout=0.1,return_sequences=True,input_shape=[None,25]),
    keras.layers.LSTM(128,recurrent_dropout=0.1),
    keras.layers.Dense(120,activation='softmax')
])
model.compile(loss='categorical_crossentropy',optimizer="adam",metrics=['accuracy'])
history=model.fit(train,epochs=50,validation_data=cv)
# model.save('lstm128_30.h5')

Epoch 1/50
926/926 [==============================] - 19s 17ms/step - loss: 4.3838 - accuracy: 0.0385 - val_loss: 4.5981 - val_accuracy: 0.0261
Epoch 2/50
926/926 [==============================] - 15s 17ms/step - loss: 4.3553 - accuracy: 0.0409 - val_loss: 4.5983 - val_accuracy: 0.0261
Epoch 3/50
926/926 [==============================] - 15s 16ms/step - loss: 4.3438 - accuracy: 0.0420 - val_loss: 4.5990 - val_accuracy: 0.0272
Epoch 4/50
926/926 [==============================] - 15s 16ms/step - loss: 4.3295 - accuracy: 0.0413 - val_loss: 4.6032 - val_accuracy: 0.0249
Epoch 5/50
926/926 [==============================] - 15s 16ms/step - loss: 4.3209 - accuracy: 0.0425 - val_loss: 4.6009 - val_accuracy: 0.0244
Epoch 6/50
926/926 [==============================] - 15s 16ms/step - loss: 4.3201 - accuracy: 0.0400 - val_loss: 4.6239 - val_accuracy: 0.0184
Epoch 7/50
926/926 [==============================] - 15s 16ms/step - loss: 4.3227 - accuracy: 0.0411 - val_loss: 4.6237 - val_accuracy:

In [11]:
model.save('lstm128_30.h5')

In [10]:
history=model.fit(train,epochs=50,validation_data=cv)

Epoch 1/50
926/926 [==============================] - 16s 17ms/step - loss: 3.2221 - accuracy: 0.2118 - val_loss: 5.7874 - val_accuracy: 0.0129
Epoch 2/50
926/926 [==============================] - 16s 17ms/step - loss: 3.1905 - accuracy: 0.2152 - val_loss: 5.7904 - val_accuracy: 0.0166
Epoch 3/50
926/926 [==============================] - 16s 17ms/step - loss: 3.1672 - accuracy: 0.2224 - val_loss: 5.8213 - val_accuracy: 0.0123
Epoch 4/50
926/926 [==============================] - 16s 17ms/step - loss: 3.1429 - accuracy: 0.2237 - val_loss: 5.8675 - val_accuracy: 0.0146
Epoch 5/50
926/926 [==============================] - 16s 17ms/step - loss: 3.1174 - accuracy: 0.2280 - val_loss: 5.8772 - val_accuracy: 0.0126
Epoch 6/50
926/926 [==============================] - 16s 17ms/step - loss: 3.0993 - accuracy: 0.2302 - val_loss: 5.9014 - val_accuracy: 0.0152
Epoch 7/50
926/926 [==============================] - 18s 19ms/step - loss: 3.0775 - accuracy: 0.2337 - val_loss: 5.9166 - val_accuracy:

In [11]:
model=keras.models.load_model('lstm128_30.h5')
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, None, 128)         81408     
_________________________________________________________________
lstm_1 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dense (Dense)                (None, 120)               15480     
Total params: 228,472
Trainable params: 228,472
Non-trainable params: 0
_________________________________________________________________


In [12]:
model=keras.models.load_model('lstm128_30.h5')
def predictions(arr):
    embed_df=pd.read_csv('hero_to_embedding.csv')
    embed_df.set_index('Heroes',inplace=True)
    instance=[]
    label=embed_df.loc[arr].index[-1]
    for i in embed_df.loc[arr].index:
        instance.append(embed_df.loc[i][embed_df.columns].values)
    print('First 4 picks are')
    print(' ')
    for i,j in enumerate(embed_df.loc[arr].index[:-1]):
        print(str(i+1)+'.'+j)
    instance=np.array(instance)
    print(len(instance))
    instance=preds(instance)
    predictions=model.predict(instance)
    ind=np.argpartition(predictions[0],-3)[-3:][::-1]
    print(' ')
    print('Suggestions are')
    
    print(' ')
    for i,j in enumerate(ind):
        print(str(i+1)+'.'+hero_dict[j])
    print(' ')
    
    print('Actual Pick')
    print(' ')
    
    print(label)

     

In [15]:
predictions(names_array[102])

First 4 picks are
 
1.Shadow Demon
2.Weaver
3.Viper
4.Luna
5.Tusk
6.Tiny
7.Elder Titan
8.Ember Spirit
9.Puck
10
 
Suggestions are
 
1.Arc Warden
2.Dark Seer
3.Wraith King
 
Actual Pick
 
Arc Warden


In [25]:
names_array[1]

array(['Phoenix', 'Beastmaster', 'Vengeful Spirit', 'Drow Ranger', 'Tiny'],
      dtype='<U19')

In [17]:
def accuracy_computing(data):
    correct=0
    for i in data:
        instance=[]
        suggestions=[]
        label=embed_df.loc[i].index[-1]
        for i in embed_df.loc[i].index:
            instance.append(embed_df.loc[i][embed_df.columns].values)
        instance=np.array(instance)
        instance=preds(instance)
        predictions=model.predict(instance)
        ind=np.argpartition(predictions[0],-3)[-3:][::-1]
        for j in ind:
            suggestions.append(hero_dict[j])
        if label in suggestions:
            correct+=1
    print("Accuracy is ",correct/data.shape[0])

In [19]:
accuracy_computing(names_array[cv_start:test_start])

Accuracy is  0.045024376254660166


In [4]:
embed_df.head()

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
Heroes,,,,,,,,,,,,,,,,,,,,,
Anti-Mage,0.305056,-0.805489,1.121158,-0.141986,0.946794,1.347950,0.258835,-0.113208,0.465853,-0.427102,...,0.784361,-0.060602,0.200479,0.327247,-0.568588,0.301570,0.169598,0.386424,0.607690,0.669263
Axe,0.228642,0.858149,-0.514170,0.602047,-0.464758,-0.336470,-0.042451,-0.074548,-0.046774,-0.557308,...,0.954994,0.601367,0.595055,-0.545509,-0.745396,0.616176,1.081884,0.190573,0.105106,0.219873
Bane,0.434433,0.015649,0.842300,-0.048546,-0.350595,-0.278436,-0.284187,1.025200,-0.605541,0.202381,...,0.625897,-0.522461,0.230606,-0.269472,-0.183916,-0.782461,-0.137054,-0.172839,-0.522343,-0.241623
Bloodseeker,0.481227,0.338672,0.370381,0.220712,-0.231627,-0.272621,0.326090,-0.009944,0.052213,-0.211072,...,0.968901,0.223838,-0.256637,-0.106540,-0.468865,0.310433,0.608026,0.157200,-0.029146,0.099427
Crystal Maiden,0.465281,-0.219395,0.984966,-0.343826,-0.423050,-0.243807,-0.128838,1.024566,-0.758725,0.398168,...,0.616749,-0.504094,-0.044536,-0.062153,-0.015833,-0.796932,-0.519311,0.098114,-0.462588,-0.265883
